In [153]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [197]:
_df = pd.read_csv('../../Datasetv2.csv')  
_df.columns

Index(['Gender', 'Birthday', 'Age', 'Order', 'Order Date', 'Trip Start Date',
       'Year', 'N Nights', 'Trip Type', 'Order Type', 'Country', 'Region',
       'N Tourists', 'Adult Tourists', 'Kid Tourists', 'Baby Tourists',
       'Star Rate', 'Group Type', 'Price', 'Trip Name', 'Client ID',
       'Booking Depth', 'Meal Option', 'Destination City', 'Departure City'],
      dtype='object')

In [198]:
df = pd.read_csv('../../Datasetv2+.csv')  

In [199]:
# df.drop(columns=['Unnamed: 0'], inplace=True)

In [200]:
df.columns

Index(['Unnamed: 0', 'Gender', 'Birthday', 'Age', 'Order', 'Order Date',
       'Trip Start Date', 'Year', 'N Nights', 'Trip Type', 'Order Type',
       'Country', 'Region', 'N Tourists', 'Adult Tourists', 'Kid Tourists',
       'Baby Tourists', 'Star Rate', 'Group Type', 'Price', 'Trip Name',
       'Client ID', 'Booking Depth', 'Meal Option', 'Destination City',
       'Departure City', 'MGRS_to', 'currency_to', 'flag_to', 'roadinfo_to',
       'timezone_name_to', 'timezone_shortname_to', 'continent_to', 'city_to',
       'country_to', 'confidence_to', 'lat_to', 'lng_to', 'MGRS_from',
       'currency_from', 'flag_from', 'roadinfo_from', 'timezone_name_from',
       'timezone_shortname_from', 'continent_from', 'city_from',
       'country_from', 'confidence_from', 'lat_from', 'lng_from'],
      dtype='object')

In [201]:
import nltk

def extract_entity_names(t):
    entity_names = []

    if hasattr(t, 'label') and t.label:
        if t.label() == 'NE':
            entity_names.append(' '.join([child[0] for child in t]))
        else:
            for child in t:
                entity_names.extend(extract_entity_names(child))

    return entity_names

In [202]:
orig_lines = df['Trip Name'].value_counts().index.to_numpy()

In [203]:
new_lines = []
for line in orig_lines:
    s = ''.join([i for i in line if not i.isdigit()])
    new_lines.append(s)

In [204]:
import re
new_new_lines = []

for line in new_lines:
    result = re.sub("[\(\[].*?[\)\]]", "", line)
    new_new_lines.append(result)

In [205]:
lines = [line.replace('TUI', '').replace('Premium', '').replace('SL', '').replace('Prestige', '').replace('.', ' ').replace('-', ' ').replace('+', ' ').replace('*', ' ').replace('RU', '').replace('MOW', '').replace('GDS', '') for line in new_new_lines]

In [206]:
tour_name_map_dict = {}
for orig_line, line in zip(orig_lines, lines):
    sentences = nltk.sent_tokenize(line)
    tokenized_sentences = [nltk.word_tokenize(sentence) for sentence in sentences]
    tagged_sentences = [nltk.pos_tag(sentence) for sentence in tokenized_sentences]
    chunked_sentences = nltk.ne_chunk_sents(tagged_sentences, binary=True)

    entities = []
    for tree in chunked_sentences:
        entities.extend(extract_entity_names(tree))
    if len(entities) > 0:
        if 0 < len(entities[0].split()) < 3:
            print(orig_line, '#', entities[0].split()[-1])
            tour_name_map_dict[orig_line] = entities[0].split()[-1]
        elif len(entities[0].split()) >= 3:
            print(orig_line, '##', entities)
            tour_name_map_dict[orig_line] = entities[0].split()[1]

Turkey Antalya MOW # Antalya
Turkey Dalaman MOW # Dalaman
Cyprus MOW # Cyprus
Turkey Antalya LED # Antalya
Cyprus S18 MOW # Cyprus
Montenegro MOW # Montenegro
Spain Barcelona MOW W19/20 # Barcelona
Turkey Bodrum MOW # Bodrum
Россия Сочи MOW GDS # Сочи
TUI Premium Antalya S19 MOW # Antalya
Tunisia MOW S2019 # Tunisia
Jordan Aqaba MOW # Aqaba
TUI Premium Antalya MOW # Antalya
Thailand Phuket W1920 MOW # Phuket
UAE Dubai MOW DWC/RKT Ural Airlines # Dubai
Greece MOW # Greece
Greece Heraklion MOW # Heraklion
Spain Mallorca MOW old # Mallorca
Россия (только отель) # Россия
Turkey Antalya KZN # Antalya
Croatia Pula MOW # Pula
Cyprus MOW GDS # Cyprus
Turkey (hotel only) RU Antalya # Antalya
Tunisia MOW S2018 # Tunisia
Italy Rimini MOW # Rimini
Bulgaria Burgas MOW BOJ # Burgas
Абхазия MOW GDS # Абхазия
UAE Dubai DWC Ural Airlines # Dubai
TUI Prestige Antalya S18 MOW # Antalya
*Индивидуальный туризм # Индивидуальный
Bulgaria Burgas S19 MOW # Burgas
Greece Heraklion S18 MOW part 2 # Heraklion
Bul

Greece Thessaloniki S18 MOW # Thessaloniki
China Hainan MOW # Hainan
UAE Dubai DXB EK 132 [БЛОКИ МЕСТ] part 2 # Dubai
Абхазия UFA GDS # UFA
Абхазия KUF GDS # KUF
SL TOUR GREECE MOW # TOUR
Spain Ibiza MOW GDS # Ibiza
Россия Сочи TJM GDS # Сочи
Germany Munich MOW GDS # Munich
Калабрия без группого трансфера S18 MOW old # Калабрия
SL TOUR Tunisia MOW # Tunisia
UAE Dubai MRV DXB GDS # Dubai
Калининград отели на побережье MOW GDS # Калининград
TUI Premium Corfu MOW # Corfu
Turkey Antalya BZK # Antalya
Italy Rome LED GDS # Rome
UAE Abu Dhabi GDS MOW AUH (Early c/out) ## ['UAE Abu Dhabi']
Italy Rimini MOW (tickets) RU # Rimini
SL TOUR Larnaca MOW + Hotel Only ## ['TOUR Larnaca Hotel']
Россия Сочи OVB GDS # Сочи
Turkey S20 (hotel only) RU Dalaman # Turkey
Россия Сочи KJA GDS # Сочи
UAE Dubai ROV DWC # Dubai
SL TOUR Rhodos MOW # Rhodos
Turkey Antalya S19 VOZ-AYT-MOW (возврат в Москву) # Antalya
Thailand Pattaya VVO GDS (S7) # Pattaya
UAE W17/18 UFA # UAE
Italy Turin SKI GDS MOW # Turin
Калининг

Turkey Antalya KZN-AYT-MOW (Возврат в Москва) # Antalya
Алтай экскурсионные туры GDS MOW-RGK # Алтай
Albania MOW TIA GDS # Albania
Южная сказка Италии S18 MOW # Южная
UAE Dubai KZN DWC part 3 # Dubai
о.Бали (10 ночей), Singapore Airlines # Airlines
UAE Dubai GDS GRV DXB part 3 # Dubai
Стамбул - город контрастов MOW # Стамбул
South Korea Seoul MOW GDS ## ['South Korea Seoul']
Thailand Koh Chang+ Bkk MOW GDS ## ['Thailand Koh Chang Bkk']
я_OLDПрага-Вена-Дрезден MOW GDS ## ['я_OLDПрага Вена Дрезден']
Эра продолжается ISR MOW GDS # Эра
Jordan MOW-AMM GDS # Jordan
я_OLDПрага Саксония MOW GDS # Саксония
Vietnam Phan Thiet OVB GDS ## ['Vietnam Phan Thiet']
Удивительное путешествие по изумрудному острову MOW GDS # Удивительное
По Следам Султана MOW GDS # Следам
Лондон-hotel only MOW GDS # Лондон
Sri Lanka W1718 # Lanka
Cyprus MRV-LCA (tickets) # Cyprus
Из Италии с любовью из Вероны MOW GDS # Италии
Dynamic package: Germany GDS # Dynamic
Новый год и Рождество на Золотом Кольце (Золотое кольцо Р

Рождественские португальские каникулы MOW GDS # Рождественские
Tunisia UFA # Tunisia
Вся Прибалтика на Новый Год или Рождество MOW GDS # Прибалтика
Spain Barcelona KUF # Barcelona
Turkey Dalaman MRV # Dalaman
Tunisia KUF # Tunisia
Turkey Bodrum S18 MOW (обратный вылет из Анталии) # Bodrum
Китай о.Хайнань (прилет в Санью) VVO # Китай
TUI Premium Antalya W19-20 KZN # Antalya
Greece MOW-CFU (tickets) # Greece
Thailand Pattaya MOW GDS part 2 # Pattaya
Thailand Pattaya MOW а/к Туркменистан (мин. тариф) # Pattaya
Цикличный тур «Янтарный вояж» LED GDS # Цикличный
UAE Dubai KZN DXB Ural Airlines [БЛОКИ МЕСТ] # Dubai
Лондон Триумф MOW GDS # Триумф
TUI Premium Antalya ASF # Antalya
Cyprus UFA # Cyprus
Каскадный тур по Грузии MOW GDS # Каскадный
Croatia Split MOW # Split
Spain Ibiza(only hotel) # Ibiza
Vietnam Phan Thiet W17/18 GDS ## ['Vietnam Phan Thiet']
Северная Пальмира, 5 дней (Санкт-Петербург без перелета) # Пальмира
Камчатка Light, 7 дней (Камчатка без перелета) # Light
Экскурсионный тур 

Россия КМВ CEK GDS # Россия
Искусство и вкусы S18 MOW # Искусство
Serbia Kopaonik MOW GDS # Kopaonik
Israel TLV SVX GDS # Israel
Turkey MOW-IST (tickets) # Turkey
Czech Republic Prague KUF GDS ## ['Czech Republic Prague']
Прага Саксония MOW GDS # Саксония
История и Современность ISR MOW GDS # История
Thailand Pattaya UUS GDS # Pattaya
UAE Инфотур (Марафон) # UAE
Вся Беларусь за 7 дней MOW GDS # Беларусь
Thailand HKT (tickets) VVO RU # Thailand
Мурманск Экскурсионные туры (только отель) # Экскурсионные
Прага Замки Чехии MOW GDS ## ['Прага Замки Чехии']
Thailand Phuket MOW GDS Robinson ## ['Thailand Phuket Robinson']
Суздальское княжество 2 дня (Золотое кольцо России) # Суздальское
Мир Диснея (без авиаперелета) # Диснея
Древние водопады и каньоны Карелии, 2 дня (Карелия без перелета) # Древние
Абхазия PEE GDS # PEE
GR PR and Press BAIKAL # PR
Рекламный тур в Калининград c TUI # Рекламный
Spain Costa Del Sol (only hotel) ## ['Spain Costa Del Sol']
Infotour Dominican Republic MOW # Republi

In [207]:
tour_name_map_dict['Czech Republic Prague MOW GDS'] = 'Prague'
tour_name_map_dict['UAE W17/18 VKO'] = 'Dubai'

In [208]:
df['Trip Name'] = df['Trip Name'].map(tour_name_map_dict)

In [209]:
df['Trip Name'].value_counts()

Antalya        110445
Cyprus          17727
Dalaman         10159
Dubai            6791
Montenegro       5331
                ...  
Весна               1
Древние             1
Trip                1
Суздальское         1
Большой             1
Name: Trip Name, Length: 324, dtype: int64

In [211]:
# df['Age'] = df['Age'].astype(int)
# df['Gender'] = df['Gender'].astype(str)
# df['Trip Type'] = df['Trip Type'].astype(str)
# df['Order Type'] = df['Order Type'].astype(str)
# df['Country'] = df['Country'].astype(str)
# df['Region'] = df['Region'].astype(str)
# df['Star Rate'] = df['Star Rate'].astype(str)
# df['Group Type'] = df['Group Type'].astype(str)
# df['Trip Name'] = df['Trip Name'].astype(str)
# df['Meal Option'] = df['Meal Option'].astype(str)
# df['Destination City'] = df['Destination City'].astype(str)
# df['Departure City'] = df['Departure City'].astype(str)

In [212]:
# col_names = {'Пол': 'Gender', 'День рождения': 'Birthday', 
#             'Возраст клиента': 'Age', 'Заявка № ': 'Order', 
#             'Дата бронирования': 'Order Date', 'Дата начала тура': 'Trip Start Date',
#             'Год': 'Year', 'Ночей': 'N Nights', 'Вид тура': 'Trip Type', 
#             'Тип заявки': 'Order Type', 'Страна тура': 'Country', 
#             'Регион отеля': 'Region', 'Туристов': 'N Tourists',
#             'Туристы, взрослые': 'Adult Tourists', 'Туристы, дети': 'Kid Tourists',
#             'Младенцы': 'Baby Tourists', 'Звездность': 'Star Rate', 
#             'Состав группы (Заявка) (Заявка)': 'Group Type', 'Сумма в $': 'Price', 
#             'Наименование тура': 'Trip Name', 'ИД клиента': 'Client ID', 
#             'Глубина продаж': 'Booking Depth', 'Тип питания': 'Meal Option', 
#             'Город Отеля': 'Destination City', 'Город отправления': 'Departure City'}
# df = df.rename(columns=col_names)

In [213]:
df['Birthday'] = pd.to_datetime(df['Birthday'])
df.loc[df['Birthday'].notnull(), 'Birthday'] = df.loc[df['Birthday'].notnull(), 'Birthday'].apply(lambda x: x.timestamp()) 

df['Order Date'] = pd.to_datetime(df['Order Date'])
df['Order Date'] = df['Order Date'].apply(lambda x: x.timestamp()) 

df['Trip Start Date'] = pd.to_datetime(df['Trip Start Date'])
df['Trip Start Date'] = df['Trip Start Date'].apply(lambda x: x.timestamp()) 


In [214]:
# df_meal = pd.read_excel('MEALS - code.xlsx')
# df_meal['Unnamed: 0'] = df_meal['Unnamed: 0'].astype(str)
# df_meal.set_index(['Unnamed: 0'], inplace=True)
# meal_map = df_meal['Code meal'].to_dict()
# df['Meal Option'] = df['Meal Option'].map(meal_map)

In [215]:
# df_stars = pd.read_excel('STARS - code.xlsx')
# df_stars['Unnamed: 0'] = df_stars['Unnamed: 0'].astype(str)
# df_stars.set_index(['Unnamed: 0'], inplace=True)
# stars_map = df_stars['Code'].to_dict()
# df['Star Rate'] = df['Star Rate'].map(stars_map)

In [216]:
df = df.set_index(['Client ID'])

In [218]:
thresh_min = 16 # could be smaller e.g. 16
# thresh_max = 60 
df = df.loc[df['Age'] >= thresh_min]
# df = df.loc[df['Age'] <= thresh_max]

In [219]:
df['Age'] = df['Age'].astype(int)

/home/alice/Documents/TempTravelHack/venv/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [220]:
def categorize(series):
    tmp_dict = series.value_counts(dropna=False).to_dict()
    tmp_list = sorted(tmp_dict.items(), key=lambda item: -item[1])
    map_dict = {}
    for i, item in enumerate(tmp_list):
        map_dict[item[0]] = i
    cat_series = series.map(map_dict)
    return cat_series, map_dict

In [221]:
df_new = pd.DataFrame(index=df.index)

In [222]:
df_new['Gender'], gender_map = categorize(df['Gender'])

In [223]:
df_new['Year'], year_map = categorize(df['Year'])

In [224]:
df_new['Trip Type'], trip_type_map = categorize(df['Trip Type'])

In [225]:
df_new['Order Type'], order_type_map = categorize(df['Order Type'])

In [226]:
df_new['Country'], country_map = categorize(df['Country'])

In [227]:
df.loc[df['Region'].isna(), 'Region'] = df.loc[df['Region'].isna()]['Trip Name']

/home/alice/Documents/TempTravelHack/venv/lib/python3.7/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [228]:
df_new['Region'], region_map = categorize(df['Region'])

In [229]:
df_new['Star Rate'], star_map = categorize(df['Star Rate'])

In [230]:
df_new['Group Type'], group_type_map = categorize(df['Group Type'])

In [231]:
# df['Trip Name'], trip_name_map = categorize(df['Trip Name'])

In [232]:
df_new['Meal Option'], meal_opt_map = categorize(df['Meal Option'])

In [233]:
df_new['Destination City'], dest_city_map = categorize(df['Destination City'])

In [234]:
df_new['Departure City'], dep_city_map = categorize(df['Departure City'])

In [235]:
df_new['Age'], age_map = categorize(df['Age'])

In [236]:
df_new['N Nights'], nights_map = categorize(df['N Nights'])

In [237]:
df_new['N Tourists'], ntourists_map = categorize(df['N Tourists'])

In [238]:
df_new['Adult Tourists'], adult_map = categorize(df['Adult Tourists'])

In [239]:
df_new['Baby Tourists'], baby_map = categorize(df['Baby Tourists'])

In [240]:
df_new['Kid Tourists'], kid_map = categorize(df['Kid Tourists'])

In [241]:
df_new

,Gender,Year,Trip Type,Order Type,Country,Region,Star Rate,Group Type,Meal Option,Destination City,Departure City,Age,N Nights,N Tourists,Adult Tourists,Baby Tourists,Kid Tourists
Client ID,,,,,,,,,,,,,,,,,
4197429,1,3,0,0,0,0,2,0,1,0,1,17,7,2,0,0,2
4197517,1,3,0,0,0,5,1,0,2,20,0,8,6,1,0,0,1
4197719,1,3,0,0,0,1,1,0,2,8,0,27,4,1,0,0,1
4199375,1,3,0,0,0,0,1,0,2,0,0,35,1,0,2,0,1
4208819,1,3,0,0,0,3,3,2,5,31,0,3,4,2,3,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4141955,0,3,4,0,2,62,4,1,3,80,0,16,3,0,0,0,0
4141837,0,3,0,0,0,0,2,3,1,0,1,8,6,3,2,0,0
4142747,0,3,0,0,0,13,2,1,1,19,0,29,2,0,0,0,0


In [260]:
cat_cols = (
    'currency_to', 'flag_to', 'roadinfo_to', 'timezone_name_to', 'timezone_shortname_to',
    'continent_to', 'city_to', 'country_to', 'confidence_to', 'MGRS_from', 'currency_from',
    'flag_from', 'roadinfo_from', 'timezone_name_from',
    'timezone_shortname_from', 'continent_from', 'city_from',
    'country_from', 'confidence_from'
)
reg_cols = ('lat_from', 'lng_from', 'lat_to', 'lng_to')

for col in cat_cols:
    df_new[col], col_map = categorize(df[col])
    
for col in reg_cols:
    df_new[col] = df[col]

In [262]:
df_continius = df[['Birthday', 'Order', 'Order Date', 'Trip Start Date', 'Price', 'Booking Depth']]

In [263]:
dfdf = pd.concat([df_new, df_continius], axis=1)

In [264]:
dfdf.columns.shape

(46,)

In [265]:
df.columns.shape

(49,)

In [270]:
dfdf.to_csv('DatasetV4+.csv')

In [267]:
dfdf

,Gender,Year,Trip Type,Order Type,Country,Region,Star Rate,Group Type,Meal Option,Destination City,...,lat_from,lng_from,lat_to,lng_to,Birthday,Order,Order Date,Trip Start Date,Price,Booking Depth
Client ID,,,,,,,,,,,,,,,,,,,,,
4197429,1,3,0,0,0,0,2,0,1,0,...,59.939043,30.315816,36.719216,31.568803,672537600.0,4803671,1.612731e+09,1.620086e+09,1718.64,85.136111
4197517,1,3,0,0,0,5,1,0,2,20,...,55.752044,37.618500,25.079911,55.147475,342230400.0,4803915,1.612775e+09,1.614816e+09,948.45,23.622917
4197719,1,3,0,0,0,1,1,0,2,8,...,55.752044,37.618500,25.079911,55.147475,-39052800.0,4806034,1.612814e+09,1.633738e+09,2128.15,242.176389
4199375,1,3,0,0,0,0,1,0,2,0,...,55.752044,37.618500,25.079911,55.147475,-300758400.0,4807039,1.612875e+09,1.614384e+09,993.85,17.468056
4208819,1,3,0,0,0,3,3,2,5,31,...,55.752044,37.618500,25.079911,55.147475,484704000.0,4808190,1.612892e+09,1.622938e+09,1121.02,116.265278
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4141955,0,3,4,0,2,62,4,1,3,80,...,55.752044,37.618500,25.079911,55.147475,160704000.0,4778962,1.611759e+09,1.612570e+09,1164.77,9.376389
4141837,0,3,0,0,0,0,2,3,1,0,...,59.939043,30.315816,36.719216,31.568803,335577600.0,4778919,1.611759e+09,1.619395e+09,468.29,88.384028
4142747,0,3,0,0,0,13,2,1,1,19,...,55.752044,37.618500,25.079911,55.147475,805334400.0,4779312,1.611766e+09,1.625789e+09,1008.25,162.303472


In [268]:
def ohe(df, cols):
    dfs = []
    for col in cols:
        ohe_df = pd.get_dummies(df[col])
        dfs.append(ohe_df)
    return pd.concat(dfs, axis=1)

In [269]:
dfdfdf = ohe(dfdf, ['Gender', 'Year', 'Trip Type', 'Order Type', 'Country', 'Region',
       'Star Rate', 'Group Type', 'Meal Option', 'Destination City',
       'Departure City', 'Age', 'N Nights', 'N Tourists', 'Adult Tourists',
       'Kid Tourists', 'Baby Tourists'])

In [254]:
# dfdfdf = df_feat[['Birthday', 'Order Date', 'Trip Start Date', 'Price', 'Booking Depth']]

In [255]:
dfdfdf

,0,1,0,1,2,3,4,5,0,1,...,1,2,3,4,5,6,7,0,1,2
Client ID,,,,,,,,,,,,,,,,,,,,,
4197429,0,1,0,0,0,1,0,0,1,0,...,0,1,0,0,0,0,0,1,0,0
4197517,0,1,0,0,0,1,0,0,1,0,...,1,0,0,0,0,0,0,1,0,0
4197719,0,1,0,0,0,1,0,0,1,0,...,1,0,0,0,0,0,0,1,0,0
4199375,0,1,0,0,0,1,0,0,1,0,...,1,0,0,0,0,0,0,1,0,0
4208819,0,1,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4141955,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4141837,1,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
4142747,1,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0


In [256]:
# from sklearn.manifold import TSNE
from tsnecuda import TSNE
X_embedded = TSNE(n_components=2, perplexity=1024.0, learning_rate=50.0).fit_transform(dfdfdf.to_numpy())
# X_embedded = TSNE(n_components=2).fit_transform()

ModuleNotFoundError: No module named 'tsnecuda'

In [257]:
X_embedded

NameError: name 'X_embedded' is not defined

In [ ]:
from sklearn.mixture import BayesianGaussianMixture
bgm = BayesianGaussianMixture(n_components=12, weight_concentration_prior=3).fit_predict(X_embedded)

In [ ]:
bgm

In [ ]:
# from sklearn.cluster import KMeans
# kmeans = KMeans(n_clusters=24).fit_predict(X_embedded)

In [ ]:
from sklearn.decomposition import PCA 
pca = PCA(n_components=2)
X_cat = pca.fit_transform(dfdfdf.to_numpy())

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
dfdfdf_norm = scaler.fit_transform(dfdfdf.to_numpy())
pca = PCA(n_components=2)
X_temporal = pca.fit_transform(dfdfdf_norm)

In [ ]:
# plt.scatter(X_all[:, 0], X_all[:, 1]) 

In [ ]:
plt.scatter(X_cat[:, 0], X_cat[:, 1]) 

In [ ]:
fig = plt.figure(figsize=(20, 20))
plt.scatter(X_embedded[:, 0], X_embedded[:, 1], c=bgm) 